<a href="https://colab.research.google.com/github/Benedictakel/bike_sharing_demand_/blob/main/Bike_Sharing_Demand_Data_from_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0" bokeh==2.0.1
!pip install autogluon --no-cache-dir


In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted


In [ ]:
import json
import os

# Move kaggle.json to correct location and secure it
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c bike-sharing-demand
!unzip -o bike-sharing-demand.zip


In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor

In [ ]:
# Load the training dataset with datetime parsing
train = pd.read_csv("train.csv", parse_dates=["datetime"])
print("Train DataFrame:")
display(train.head())

In [ ]:
# Load the test dataset
test = pd.read_csv("test.csv", parse_dates=["datetime"])
print("Test DataFrame:")
display(test.head())

In [ ]:
# Load the sample submission file
submission = pd.read_csv("sampleSubmission.csv")
print("Sample Submission:")
display(submission.head())

In [ ]:
predictor = TabularPredictor(
    label='count',
    eval_metric='root_mean_squared_error'
).fit(
    train_data=train.drop(columns=["casual", "registered"]),
    time_limit=600,
    presets='best_quality'
)

predictor.fit_summary()


In [ ]:
# Generate predictions
predictions = predictor.predict(test)

In [ ]:
# Check for negatives
print(predictions.describe())
print("Negative values count:", (predictions < 0).sum())

In [ ]:
# Set any negatives to 0
predictions[predictions < 0] = 0

In [ ]:
# Prepare submission
submission["count"] = predictions
submission.to_csv("submission.csv", index=False)

In [ ]:
# Submit to Kaggle
!kaggle competitions submit -c bike-sharing-demand -f submission.csv -m "first raw submission"
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6